**Lab1**
Carlos García 21000475

### Import libraries

In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime

print(tf.__version__)

2.5.0-rc3


In [19]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


### Getting the data

In [3]:
data = np.load('inputs/proyecto_training_data.npy')

In [5]:
print(data.shape)
n_train = int(np.ceil(data.shape[0]*0.8))

(1460, 6)


In [6]:
train, test = data[:n_train, :], data[n_train:, :] 
assert (train.shape[0] + test.shape[0]) == data.shape[0]

In [7]:
ds_train = pd.DataFrame(train, columns = ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage'])
ds_train.head()

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
0,208500.0,7.0,856.0,8.0,2003.0,65.0
1,181500.0,6.0,1262.0,6.0,1976.0,80.0
2,223500.0,7.0,920.0,6.0,2001.0,68.0
3,140000.0,7.0,961.0,7.0,1915.0,60.0
4,250000.0,8.0,1145.0,9.0,2000.0,84.0


In [8]:
response = 'SalePrice'
regressor = 'OverallQual'

### Defining the model

In [9]:
def error(y_real,y_aprox):
  return 1/2 * tf.reduce_mean(tf.math.square(y_real - y_aprox))

In [86]:
#mini batch model
def trainModel(x, y, epochs = 100, batch_size = 10, lr = 0.001, kprint = 10):
    
    #define iterations
    total_iterations = x.shape[0] // batch_size

    #initializing the graph
    tf.reset_default_graph()

    #initializing data
    tensor_x = tf.placeholder(tf.float32, [None, 2], "tensor_x")
    tensor_y = tf.placeholder(tf.float32, [None, 1], "tensor_y")
    w = tf.get_variable("parameters_m_b", dtype=tf.float32, shape=[2,1],
                    initializer=tf.zeros_initializer())

    #estimating values
    yhat = tf.matmul(tensor_x, w, name = 'yhat')

    #Cost/Error calculation
    with tf.name_scope("cost_definition"):
        cost = error(tensor_y, yhat)
        
    #Scalar summary
    cost_summary = tf.summary.scalar(name = 'MSE', tensor = cost)

    #gradients and cost/error optimization
    with tf.name_scope("params_update"):
        gradients = tf.gradients(cost, [w], name = 'gradients') #calculating error and gradients
        w_update = tf.assign(w, w - lr * gradients[0], name = 'weigths_update') #updating parameters weights
    
    #with tf.train.MonitoredSession() as session: #this object does'nt generate a clean graph due to initialized variables
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        #Reshaping the data
        x = x[:, np.newaxis]
        x = np.hstack((x, np.ones_like(x)))
        y = y[:, np.newaxis]

        feed_dict_model = {tensor_x: x, tensor_y: y} #whole batch dictionary

        #Define tensorboard writer and config string
        dt_string = datetime.now().strftime("%Y%m%d_%H%M")
        writer = tf.summary.FileWriter('./graphs/{}_lm_epochs={}_mbatch={}_lr={}'.format(dt_string, epochs, batch_size, lr), session.graph)
        
        for epoch in range(0, epochs):    
            for i in range(0, total_iterations):
                start_sample = i * batch_size
                end_sample = start_sample + batch_size
                x_mb = x[start_sample:end_sample]
                y_mb = y[start_sample:end_sample]
                
                feed_dict = {tensor_x: x_mb, tensor_y: y_mb}
                _, weights = session.run([w_update, w], feed_dict = feed_dict) #be careful not to use the same name of a previous variable
                #m, b = weights[0, 0], weights[1, 0]

                #print("Epoch {} iteration {} m={} b={}".format(epoch,i,m,b))

            predictions = session.run(yhat, feed_dict = feed_dict_model)
            mse, csummary = session.run([cost, cost_summary], feed_dict = feed_dict_model)
            writer.add_summary(csummary, epoch + 1)

            if (epoch + 1) % kprint == 0:            
                m, b = weights[0, 0], weights[1, 0]
                print("Epoch {} parameters: m={} b={} mse={}".format(epoch + 1, m, b, mse))

        _, weights = session.run([w_update, w], feed_dict = feed_dict_model)
        predictions, mse = session.run([yhat, cost], feed_dict = feed_dict_model)
        m, b = weights[0, 0], weights[1, 0]
        print("Final model parameters: m={} b={} mse={}".format(m,b,mse))

        writer.close()
            

### Graph definition

<img src="imgs/graph_definition.png">

### Experiments

In [100]:
#Experiment1. Initial experiment, expecting MSE going down. This experiment will work as a starting point to tune next experiments. Small lr and epochs
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 100, batch_size = 250, lr = 0.01, kprint = 10 )

Epoch 10 parameters: m=30116.4921875 b=2648.34814453125 mse=1385676160.0
Epoch 20 parameters: m=30420.65625 b=697.010986328125 mse=1376387200.0
Epoch 30 parameters: m=30719.11328125 b=-1217.7012939453125 mse=1367451648.0
Epoch 40 parameters: m=31011.96875 b=-3096.478271484375 mse=1358856704.0
Epoch 50 parameters: m=31299.330078125 b=-4939.99365234375 mse=1350589696.0
Epoch 60 parameters: m=31581.294921875 b=-6748.91015625 mse=1342637952.0
Epoch 70 parameters: m=31857.96875 b=-8523.8759765625 mse=1334989568.0
Epoch 80 parameters: m=32129.44921875 b=-10265.53125 mse=1327633536.0
Epoch 90 parameters: m=32395.8359375 b=-11974.498046875 mse=1320558080.0
Epoch 100 parameters: m=32657.22265625 b=-13651.390625 mse=1313753472.0
Final model parameters: m=32594.11328125 b=-13701.400390625 mse=1313203584.0


Good starting point, MSE is going down at a good peace but too few epochs implemented

In [90]:
#Experiment2. Since the MSE decayed really quickly will only add more epochs. This will keep lowering it down
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 250, lr = 0.01, kprint = 100)

Epoch 100 parameters: m=32657.22265625 b=-13651.390625 mse=1313753472.0
Epoch 200 parameters: m=35015.90625 b=-28783.232421875 mse=1258518272.0
Epoch 300 parameters: m=36967.48046875 b=-41303.3125 mse=1221211904.0
Epoch 400 parameters: m=38582.2109375 b=-51662.41796875 mse=1196092416.0
Epoch 500 parameters: m=39918.2421875 b=-60233.546875 mse=1179243264.0
Epoch 600 parameters: m=41023.671875 b=-67325.3046875 mse=1167995648.0
Epoch 700 parameters: m=41938.30859375 b=-73193.03125 mse=1160533888.0
Epoch 800 parameters: m=42695.07421875 b=-78047.9765625 mse=1155622016.0
Epoch 900 parameters: m=43321.234375 b=-82065.0078125 mse=1152422272.0
Epoch 1000 parameters: m=43839.3046875 b=-85388.6328125 mse=1150366592.0
Final model parameters: m=43778.48828125 b=-85403.4921875 mse=1150052608.0


As expected initial hyper parameters work fine, increasing epochs shows a good progress

In [91]:
#Experiment 3. Experiment with a higher batch size may cause a better performance, so batch_size is increasing to 500
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 500, lr = 0.01, kprint = 100)

Epoch 100 parameters: m=31320.9453125 b=-4932.8759765625 mse=1350775296.0
Epoch 200 parameters: m=32675.421875 b=-13640.44921875 mse=1313930752.0
Epoch 300 parameters: m=33907.5234375 b=-21561.31640625 mse=1283609856.0
Epoch 400 parameters: m=35028.3125 b=-28766.544921875 mse=1258671360.0
Epoch 500 parameters: m=36047.83984375 b=-35320.82421875 mse=1238173184.0
Epoch 600 parameters: m=36975.25390625 b=-41282.921875 mse=1221337216.0
Epoch 700 parameters: m=37818.875 b=-46706.34375 mse=1207519616.0
Epoch 800 parameters: m=38586.28125 b=-51639.78125 mse=1196189696.0
Epoch 900 parameters: m=39284.35546875 b=-56127.52734375 mse=1186908800.0
Epoch 1000 parameters: m=39919.359375 b=-60209.7890625 mse=1179315200.0
Final model parameters: m=39855.8515625 b=-60237.28125 mse=1178926208.0


A too high batch size is not working since MSE is going up again

In [92]:
#Experiment 4. Higher batchsize didn't work out, lowering it down a little more to check for optimal hyper parameters, expect better performance
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 200, lr = 0.01, kprint = 100)

Epoch 100 parameters: m=33271.0859375 b=-17741.33203125 mse=1297591040.0
Epoch 200 parameters: m=36031.0546875 b=-35391.578125 mse=1237790976.0
Epoch 300 parameters: m=38208.921875 b=-49319.2265625 mse=1201270144.0
Epoch 400 parameters: m=39927.4609375 b=-60309.42578125 mse=1179094144.0
Epoch 500 parameters: m=41283.55078125 b=-68981.7421875 mse=1165730432.0
Epoch 600 parameters: m=42353.6484375 b=-75825.1015625 mse=1157760128.0
Epoch 700 parameters: m=43198.046875 b=-81225.109375 mse=1153074944.0
Epoch 800 parameters: m=43864.35546875 b=-85486.2109375 mse=1150376064.0
Epoch 900 parameters: m=44390.12890625 b=-88848.5625 mse=1148868224.0
Epoch 1000 parameters: m=44805.015625 b=-91501.796875 mse=1148065536.0
Final model parameters: m=44739.3828125 b=-91514.4453125 mse=1147707904.0


Lowering batch size a bit more makes an improvement without consuming too much additional time

In [93]:
#Experiment 5. Lower batch size even more to check time-performance tradeoff, expecting poor performance
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 50, lr = 0.01, kprint = 100)

Epoch 100 parameters: m=39151.5234375 b=-63683.34765625 mse=1198726784.0
Epoch 200 parameters: m=42793.3515625 b=-86464.453125 mse=1171249792.0
Epoch 300 parameters: m=44011.484375 b=-94084.390625 mse=1167746048.0
Epoch 400 parameters: m=44418.96484375 b=-96633.3515625 mse=1167209984.0
Epoch 500 parameters: m=44555.234375 b=-97485.78125 mse=1167102080.0
Epoch 600 parameters: m=44600.8203125 b=-97770.9140625 mse=1167074304.0
Epoch 700 parameters: m=44616.05859375 b=-97866.2578125 mse=1167065600.0
Epoch 800 parameters: m=44621.203125 b=-97898.4296875 mse=1167062784.0
Epoch 900 parameters: m=44622.78515625 b=-97908.3046875 mse=1167061888.0
Epoch 1000 parameters: m=44623.44140625 b=-97912.4296875 mse=1167061504.0
Final model parameters: m=45019.953125 b=-97849.890625 mse=1154176768.0


A too small batch size converge faster than other without leaving space to improvement

In [94]:
#Experiment 6. Returning to best performance settings on experiment 4 with a lower learning rate, this would improve the model
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 200, lr = 0.001, kprint = 100)

Epoch 100 parameters: m=30205.626953125 b=2158.34619140625 mse=1383410688.0
Epoch 200 parameters: m=30582.224609375 b=-262.24005126953125 mse=1371967616.0
Epoch 300 parameters: m=30950.0078125 b=-2626.20361328125 mse=1361066880.0
Epoch 400 parameters: m=31309.193359375 b=-4934.87060546875 mse=1350682112.0
Epoch 500 parameters: m=31659.974609375 b=-7189.5263671875 mse=1340790144.0
Epoch 600 parameters: m=32002.5546875 b=-9391.4501953125 mse=1331367424.0
Epoch 700 parameters: m=32337.111328125 b=-11541.8642578125 mse=1322392704.0
Epoch 800 parameters: m=32663.853515625 b=-13641.978515625 mse=1313843968.0
Epoch 900 parameters: m=32982.94921875 b=-15692.974609375 mse=1305701504.0
Epoch 1000 parameters: m=33294.578125 b=-17695.984375 mse=1297946752.0
Final model parameters: m=33288.01171875 b=-17700.830078125 mse=1297881088.0


A too small learning rate makes it time consuming to optimize the cost function

In [95]:
#Experiment 7. Lower learning rate was too slowly decaying, trying more epochs to check if it's going down
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 2000, batch_size = 200, lr = 0.001, kprint = 100)

Epoch 100 parameters: m=30205.626953125 b=2158.34619140625 mse=1383410688.0
Epoch 200 parameters: m=30582.224609375 b=-262.24005126953125 mse=1371967616.0
Epoch 300 parameters: m=30950.0078125 b=-2626.20361328125 mse=1361066880.0
Epoch 400 parameters: m=31309.193359375 b=-4934.87060546875 mse=1350682112.0
Epoch 500 parameters: m=31659.974609375 b=-7189.5263671875 mse=1340790144.0
Epoch 600 parameters: m=32002.5546875 b=-9391.4501953125 mse=1331367424.0
Epoch 700 parameters: m=32337.111328125 b=-11541.8642578125 mse=1322392704.0
Epoch 800 parameters: m=32663.853515625 b=-13641.978515625 mse=1313843968.0
Epoch 900 parameters: m=32982.94921875 b=-15692.974609375 mse=1305701504.0
Epoch 1000 parameters: m=33294.578125 b=-17695.984375 mse=1297946752.0
Epoch 1100 parameters: m=33598.91796875 b=-19652.140625 mse=1290561152.0
Epoch 1200 parameters: m=33896.13671875 b=-21562.51953125 mse=1283527552.0
Epoch 1300 parameters: m=34186.40234375 b=-23428.22265625 mse=1276829184.0
Epoch 1400 parameters

Definitely going down, but taking too much time in comparison with other models

In [96]:
#Experiment 8. Lower learning rate is too slow, changing to best params (experiment 4) but increasing learning rate expect better performance
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 200, lr = 0.1, kprint = 100)

Epoch 100 parameters: m=nan b=nan mse=nan
Epoch 200 parameters: m=nan b=nan mse=nan
Epoch 300 parameters: m=nan b=nan mse=nan
Epoch 400 parameters: m=nan b=nan mse=nan
Epoch 500 parameters: m=nan b=nan mse=nan
Epoch 600 parameters: m=nan b=nan mse=nan
Epoch 700 parameters: m=nan b=nan mse=nan
Epoch 800 parameters: m=nan b=nan mse=nan
Epoch 900 parameters: m=nan b=nan mse=nan
Epoch 1000 parameters: m=nan b=nan mse=nan
Final model parameters: m=nan b=nan mse=nan


A too high learning rate crash the model, it may be related to divergence

In [97]:
#Experiment 9. Slightly increasing mini batch size to improve performance from experiment 4
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 1000, batch_size = 300, lr = 0.01, kprint = 100)

Epoch 100 parameters: m=32346.158203125 b=-9998.3603515625 mse=1331691264.0
Epoch 200 parameters: m=34326.375 b=-22744.3515625 mse=1282051968.0
Epoch 300 parameters: m=36046.53125 b=-33816.4296875 mse=1245328512.0
Epoch 400 parameters: m=37540.78125 b=-43434.42578125 mse=1218254976.0
Epoch 500 parameters: m=38838.79296875 b=-51789.3046875 mse=1198379520.0
Epoch 600 parameters: m=39966.33984375 b=-59046.94140625 mse=1183862912.0
Epoch 700 parameters: m=40945.8125 b=-65351.46875 mse=1173326848.0
Epoch 800 parameters: m=41796.640625 b=-70827.9765625 mse=1165739520.0
Epoch 900 parameters: m=42535.73046875 b=-75585.25 mse=1160329216.0
Epoch 1000 parameters: m=43177.7734375 b=-79717.8515625 mse=1156520704.0
Final model parameters: m=43029.7734375 b=-79749.0546875 mse=1154690304.0


Batch size should not increase any farther, it's value seems optimal for the model

In [98]:
#Experiment 10. Minibatch size set, increasing epochs to tune best model and check if improvement is made, expecting lower improvement
trainModel(ds_train[regressor].values, ds_train[response].values, epochs = 2000, batch_size = 200, lr = 0.01, kprint = 200)

Epoch 200 parameters: m=36031.0546875 b=-35391.578125 mse=1237790976.0
Epoch 400 parameters: m=39927.4609375 b=-60309.42578125 mse=1179094144.0
Epoch 600 parameters: m=42353.6484375 b=-75825.1015625 mse=1157760128.0
Epoch 800 parameters: m=43864.35546875 b=-85486.2109375 mse=1150376064.0
Epoch 1000 parameters: m=44805.015625 b=-91501.796875 mse=1148065536.0
Epoch 1200 parameters: m=45390.74609375 b=-95247.609375 mse=1147513088.0
Epoch 1400 parameters: m=45755.4609375 b=-97579.984375 mse=1147513088.0
Epoch 1600 parameters: m=45982.55078125 b=-99032.234375 mse=1147646720.0
Epoch 1800 parameters: m=46123.94921875 b=-99936.484375 mse=1147781376.0
Epoch 2000 parameters: m=46212.0 b=-100499.578125 mse=1147885312.0
Final model parameters: m=46144.9140625 b=-100508.0859375 mse=1147519744.0


Converge was reach around 1000 epochs, so a higher epoch is not needed

### Models results

<img src="imgs/ModelsOutput.png">

### Conclusions

- Hyper parameter tunning makes a big difference in model performance. As shown in the graph **experiment number 4** was the best in time-performance evaluation. 
- Experiment 4 learning rate was small enough to allow a fast optimization of the cost function (MSE) without taking too much time and not allowing for divergence. 
- Epochs do help optimizing but we should check for the point where convergence starts to avoid training more than needed
- Mini batch size has a sweet point between small and high, a too high batch size is not so useful, and a lower one makes training longer